In [103]:
import random
n=6
V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
import matplotlib.pyplot as plt
alpha=2

In [111]:
V

[(59.60230863891071, 44.84521093306686),
 (15.957642824352993, 5.584901238283781),
 (15.92129440962412, 52.79258871267679),
 (21.137558934740103, 25.886845591574666),
 (49.267143783148356, 51.655431819995705),
 (10.85424385943711, 31.490148063297152)]

In [104]:
alpha=2
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return (1/alpha)*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))

In [105]:
def tpath_pathcost(start,end,tpath):
    pathcost=0
    a=tpath.index(start)
    b=tpath.index(end)
    if a<b:
        for i in range(a,b):
            pathcost=pathcost+t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])
    if a>b:
        for i in range(a,len(tpath)):
            pathcost=pathcost+t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])
        for i in range(b):
            pathcost=pathcost+t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])
    if a==b:
        pathcost=0
            
    return pathcost

In [106]:
def center(V):
    x=[]
    y=[]
    for i in range(n):
        x.append(V[i][0])
        y.append(V[i][1])
    return [sum(x)/n,sum(y)/n]

In [107]:
import itertools
import matplotlib.pyplot as plt
def two_opt_truck(ps,truck_nodes):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
    path=[truck_nodes[i] for i in res]

    return path

In [108]:
def dp_truck(ps,truck_nodes):
    length = {} # length[(u,S)]: u を始点とし S の点すべてを回る最小経路長、ディクショナリ
    route = {} # route[(u,S)]: 最小経路長を達成するためのルート、ディクショナリ
    v = ps[0]
    n = len(ps)
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S = frozenset(a)
            for j in S:
                u = ps[j]
                if i==1:
                    length[(j,S)] = t_cost(v,u)
                    route[(j,S)] = [j]
                else:
                    Sj = S-set([j]) # S から j を除いたもの
                    k=min(Sj,key=lambda k: length[(k,Sj)]+t_cost(ps[k],u))#Sjの中でlength[(k,Sj)]+dist(ps[k],u)が最小のものを返している
                    length[(j,S)] = length[(k,Sj)]+t_cost(ps[k],u)
                    route[(j,S)] = route[(k,Sj)]+[j]
    path=[truck_nodes[i] for i in route[(0,frozenset(range(n)))]]
    
    return path 

In [116]:
#トラックのルートをdp（動的計画）で求める
import itertools
def newalgo_dp_issho(V):
    n=len(V)
    ndmin=2
    ndmax=n//2
    c=center(V)
    D={}
    for nd in range(ndmin,ndmax+1):
        length={i:t_cost(c,V[i]) for i in range(n)}
        drone_nodes=[]
        truck_nodes=[]
        T={}
        for i in range(nd):
            drone_nodes.append(max(length, key=length.get))
            del length[max(length, key=length.get)]
        truck_nodes=list(set(range(n))-set(drone_nodes))
        t=[V[i] for i in truck_nodes]
        tpath=dp_truck(t,truck_nodes)
        for drone in drone_nodes:
            for (start,end) in itertools.permutations(tpath,2):
                drone_cost=d_cost(V[start],V[drone])+d_cost(V[drone],V[end])
                truck_cost=tpath_pathcost(start,end,tpath)
                T[start,drone,end]=max(drone_cost,truck_cost)
        for i in range(len(tpath)):
            T[tpath[i],-1,tpath[(i+1)%len(tpath)]]=t_cost(V[tpath[i]],V[tpath[(i+1)%len(tpath)]])

        tc=[a for a in itertools.combinations(tpath,nd)]
        dc=[b for b in itertools.permutations(drone_nodes,nd)]
        for comb in itertools.product(tc,dc):
            d=0
            for j in range(nd):
                d=d+T[comb[0][j],comb[1][j],comb[0][(j+1)%nd]]
            D[comb]=d,tpath
        if len(tpath)>len(drone_nodes):
            for k in range(len(tpath)):
                if k==0:
                    between=tpath[k+1:k+-1]
                else:
                    between=tpath[k+1:k-1]+tpath[k+1:]+tpath[:k-1]
                for g in itertools.combinations(between,nd-1):
                    combine=[tpath[k],tpath[k-1]]
                    for j in g:
                        combine.insert(-1,j)
                    d=T[tpath[k-1],-1,tpath[k]]
                    for drone in dc:
                        comb=(tuple(combine),tuple(drone))
                        for j in range(nd):
                            d=d+T[combine[j],drone[j],combine[j+1]]
                        D[comb]=d,tpath
                            
                
                
                
    res=min(D.values()) 
    path=min(D,key=D.get)
        
        
    return res,path,D

In [117]:
newalgo_dp_issho(V)

105.12115497107885

In [49]:
g=(3,0)

In [51]:
tu=(1,g,4)

In [52]:
tu

(1, (3, 0), 4)

In [86]:
lis=[0,1,2,3,4,5,6]

In [87]:
lis[1:-1]

[1, 2, 3, 4, 5]

In [95]:
for k in range(len(lis)):
    if k==0:
        between=lis[k+1:k-1]
        print(between)
    else:
        between=lis[k+1:k-1]+lis[k+1:]+lis[:k-1]
        print(between)

[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 0]
[4, 5, 6, 0, 1]
[5, 6, 0, 1, 2]
[6, 0, 1, 2, 3]
[0, 1, 2, 3, 4]
